# Best poet 

<p style="font-size: 20px; font-weight: bold; font-style: italic;">...via LLM-graph</p>

Anton Antonov   
August 2025

---

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph using different LLM personas representing four poets and a poem judge.
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

(The poet-judge examples are inspired from the Wolfram Language function page of [`LLMGraph`](https://reference.wolfram.com/language/ref/LLMGraph.html), [WRIf1].)

----

## Setup

In [1]:
use LLM::Graph;
use Graph;
use LLM::Functions;

In [2]:
my $engine = 'neato';

neato

-----

## LLM graph

Nodes spec:

In [3]:
sink my %rules =
    poet1 => "Write a short poem about summer.",
    poet2 => "Write a haiku about winter.",
    poet3 => sub ($topic, $style) {"Write a poem about $topic in the style $style."},
    judge => sub ($poet1, $poet2, $poet3) {
        ["Choose the best composition of your among these:\n\n",
        "1) Poem1: $poet1",
        "2) Poem2: $poet2",
        "3) Poem3: $poet3", 
        "and copy it:"].join("\n\n") }
;

More complicated spec:

In [4]:
sink my %rules =
        poet1 => "Write a short poem about summer.",
        poet2 => "Write a haiku about winter.",
        poet3 => sub ($topic, $style) {
            "Write a poem about $topic in the $style style."
        },
        poet4 => {
                eval-function => {llm-synthesize('You are a famous Russian poet. Write a short poem about playing bears.')},
                test-function => -> $with-russian { $with-russian ~~ Bool:D && $with-russian || $with-russian.Str.lc ∈ <true yes> }
        },
        judge => sub ($poet1, $poet2, $poet3, $poet4) {
            [
                "Choose the composition you think is best among these:\n\n",
                "1) Poem1: $poet1",
                "2) Poem2: $poet2",
                "3) Poem3: {$poet4.defined && $poet4 ?? $poet4 !! $poet3}",
                "and copy it:"
            ].join("\n\n")
        };

Make the graph:

In [5]:
my $gBestPoem = LLM::Graph.new(%rules)

LLM::Graph(size => 5, nodes => judge, poet1, poet2, poet3, poet4)

----

## Full computation

In [6]:
$gBestPoem.eval(topic => 'Hockey', style => 'limerick', with-russian => 'yes')

LLM::Graph(size => 5, nodes => judge, poet1, poet2, poet3, poet4)

In [7]:
#% html
$gBestPoem.dot(engine => 'dot', :9graph-size, node-width => 1.2, node-fill=>'steelblue'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 
 judge_Cluster 
 
 
 
 poet3_Cluster 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- poet4 -->
 
 poet4 
 
 poet4 
 
<!-- $_->poet4 -->
 
 $_->poet4 
 
 
 
<!-- judge -->
 
 judge 
 
 judge 
 
<!-- poet1 -->
 
 poet1 
<polygon fill="SteelBlue" stroke="Black" points="264.04,-73.63 266.86,-73.69 269.65,-73.78 272.4,-73.89 275.09,-74.04 277.73,-74.22 280.29,-74.42 282.77,-74.65 285.15,-74.91 287.44,-75.19 289.61,-75.5 291.67,-75.83 293.61,-76.18 295.41,-76.56 297.08,-76.96 298.61,-77.38 300,-77.82 301.24,-78.28 302.33,-78.76 303.27,-79.24 304.06,-79.75 304.7,-80.27 305.19,-80.79 305.53,-81.33 305.72,-81.88 305.78,-82.43 305.69,-82.99 305.47,-83.55 305.11,-84.12 304.63,-84.68 304.03,-85.25 303.32,-85.81 302.49,-86.37 301.56,-86.92 300.54,-87.47 299.42,-88.01 298.22,-88.53 296.93,-89.05 295.58,-89.56 294.16,-90.04 292.68,-90.52 291.14,-90.98 289.56,-91.42 287.93,-91.84 286.26,-92.24 284.55,-92.62 282.82,-92.97 281.05,-93.3 279.27,-93.61 277.46,-93.89 275.64,-94.15 273.81,-94.38 271.96,-94.58 270.1,-94.76 268.24,-94.91 266.36,-95.02 264.49,-95.11 262.61,-95.17 260.73,-95.2 258.85,-95.2 256.97,-95.17 255.09,-95.11 253.21,-95.02 251.34,-94.91 249.47,-94.76 247.62,-94.58 245.77,-94.38 243.93,-94.15 242.11,-93.89 240.31,-93.61 238.52,-93.3 236.76,-92.97 235.02,-92.62 233.32,-92.24 231.65,-91.84 230.02,-91.42 228.43,-90.98 226.9,-90.52 225.42,-90.04 224,-89.56 222.64,-89.05 221.36,-88.53 220.16,-88.01 219.04,-87.47 218.01,-86.92 217.08,-86.37 216.26,-85.81 215.54,-85.25 214.94,-84.68 214.46,-84.12 214.11,-83.55 213.89,-82.99 213.8,-82.43 213.85,-81.88 214.05,-81.33 214.39,-80.79 214.88,-80.27 215.52,-79.75 216.31,-79.24 217.25,-78.76 218.34,-78.28 219.58,-77.82 220.96,-77.38 222.49,-76.96 224.16,-76.56 225.97,-76.18 227.9,-75.83 229.96,-75.5 232.14,-75.19 234.42,-74.91 236.81,-74.65 239.29,-74.42 241.85,-74.22 244.48,-74.04 247.18,-73.89 249.93,-73.78 252.71,-73.69 255.53,-73.63 258.37,-73.6 261.21,-73.6 264.04,-73.63"/>
 poet1 
 
<!-- poet1->judge -->
 
 poet1->judge 
 
 
 
<!-- poet2 -->
 
 poet2 
<polygon fill="SteelBlue" stroke="Black" points="374.04,-73.63 376.86,-73.69 379.65,-73.78 382.4,-73.89 385.09,-74.04 387.73,-74.22 390.29,-74.42 392.77,-74.65 395.15,-74.91 397.44,-75.19 399.61,-75.5 401.67,-75.83 403.61,-76.18 405.41,-76.56 407.08,-76.96 408.61,-77.38 410,-77.82 411.24,-78.28 412.33,-78.76 413.27,-79.24 414.06,-79.75 414.7,-80.27 415.19,-80.79 415.53,-81.33 415.72,-81.88 415.78,-82.43 415.69,-82.99 415.47,-83.55 415.11,-84.12 414.63,-84.68 414.03,-85.25 413.32,-85.81 412.49,-86.37 411.56,-86.92 410.54,-87.47 409.42,-88.01 408.22,-88.53 406.93,-89.05 405.58,-89.56 404.16,-90.04 402.68,-90.52 401.14,-90.98 399.56,-91.42 397.93,-91.84 396.26,-92.24 394.55,-92.62 392.82,-92.97 391.05,-93.3 389.27,-93.61 387.46,-93.89 385.64,-94.15 383.81,-94.38 381.96,-94.58 380.1,-94.76 378.24,-94.91 376.36,-95.02 374.49,-95.11 372.61,-95.17 370.73,-95.2 368.85,-95.2 366.97,-95.17 365.09,-95.11 363.21,-95.02 361.34,-94.91 359.47,-94.76 357.62,-94.58 355.77,-94.38 353.93,-94.15 352.11,-93.89 350.31,-93.61 348.52,-93.3 346.76,-92.97 345.02,-92.62 343.32,-92.24 341.65,-91.84 340.02,-91.42 338.43,-90.98 336.9,-90.52 335.42,-90.04 334,-89.56 332.64,-89.05 331.36,-88.53 330.16,-88.01 329.04,-87.47 328.01,-86.92 327.08,-86.37 326.26,-85.81 325.54,-85.25 324.94,-84.68 324.46,-84.12 324.11,-83.55 323.89,-82.99 323.8,-82.43 323.85,-81.88 324.05,-81.33 324.39,-80.79 324.88,-80.27 325.52,-79.75 326.31,-79.24 327.25,-78.76 328.34,-78.28 329.58,-77.82 330.96,-77.38 332.49,-76.96 334.16,-76.56 335.97,-76.18 337.9,-75.83 339.96,-75.5 342.14,-75.19 344.42,-74.91 346.81,-74.65 349.29,-74.42 351.85,-74.22 354.48,-74.04 357.18,-73.89 359.93,-73.78 362.71,-73.69 365.53,-73.63 368.37,-73.6 371.21,-73.6 374.04,-73.63"/>
 poet2 
 
<!-- p

In [9]:
.say for |$gBestPoem.nodes<judge>

input => [poet3 poet1 poet2 poet4]
test-function-input => []
result => I think Poem 1 is the best among these for its vivid imagery and joyful tone. Here it is:

Golden rays and skies of blue,  
Whispers warm in morning dew.  
Laughter dances on the breeze,  
Summer sings through swaying trees.  

Fields of green and oceans wide,  
Endless days where dreams can glide.  
Hearts alight with sun’s bright gleam,  
Summer lives within a dream.
wrapper => Routine::WrapHandle.new
eval-function => sub { }
spec-type => (Routine)


In [10]:
#% html
$gBestPoem.nodes.nodemap(*<result>)
==> to-html(align => 'left')

poet4 В лесу тенистом, где шёпчет трава, 
Медведи играют — весна пришла. 
В лапах мох мягкий, в глазах светлый свет, 
Играют беззаботно — им радостен свет.

Они качаются на ветках, как дети, 
Гудят, словно песни, в тиши и в рассвете. 
Мир их прост и добр, словно сказка в ночи — 
Медвежьи забавы звучат в тишине ключи. judge I think Poem 1 is the best among these for its vivid imagery and joyful tone. Here it is:

Golden rays and skies of blue, 
Whispers warm in morning dew. 
Laughter dances on the breeze, 
Summer sings through swaying trees. 

Fields of green and oceans wide, 
Endless days where dreams can glide. 
Hearts alight with sun’s bright gleam, 
Summer lives within a dream. poet3 There once was a player named Jack, 
Who loved to play hockey on the track. 
He’d skate with great speed, 
And shoot to succeed, 
Scoring goals that would never look back! poet1 Golden rays and skies of blue, 
Whispers warm in morning dew. 
Laughter dances on the breeze, 
Summer sings through swaying trees. 

Fields of green and oceans wide, 
Endless days where dreams can glide. 
Hearts alight with sun’s bright gleam, 
Summer lives within a dream. poet2 Silent snowflakes fall, 
Blanketing the earth in white, 
Winter’s quiet breath.

-----

## References

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).

[WRIf1] Wolfram Research (2025), 
[LLMGraph](https://reference.wolfram.com/language/ref/LLMGraph.html), 
[Wolfram Language function](https://reference.wolfram.com/language).